# Task 1


In [1]:
val spark= SparkSession.builder().getOrCreate()
import org.apache.spark.rdd.RDD
val pagecounts = sc.textFile("/home/osboxes/lab1/pagecounts-20160101-000000_parsed.out")
print(pagecounts)
pagecounts.count()

/home/osboxes/lab1/pagecounts-20160101-000000_parsed.out MapPartitionsRDD[1] at textFile at <console>:29

spark = org.apache.spark.sql.SparkSession@5ee3f9b6
pagecounts = /home/osboxes/lab1/pagecounts-20160101-000000_parsed.out MapPartitionsRDD[1] at textFile at <console>:29


3324129

Then, convert the pagecounts from RDD[String] into RDD[Log]:

1 Create a case class called Log using the four ﬁeld names of the dataset.

In [2]:
case class Log(code: String, title:String, hits:Int, size:Long)

defined class Log


2 Create a function that takes a string, split it by white space and converts it into a log object.

In [3]:
def Log_obj(s: String) : Log ={
    val splits = s.split(" ")
    return Log(splits(0).toString,splits(1).toString,splits(2).toInt,splits(3).toLong)
}

Log_obj: (s: String)Log


3 Create a function that takes an RDD[String] and returns an RDD[Log].

In [4]:
def RDD_Log(s: RDD[String]) : RDD[Log] = {
    return s.map(x=>Log_obj(x))
}

RDD_Log: (s: org.apache.spark.rdd.RDD[String])org.apache.spark.rdd.RDD[Log]


In [5]:
val rdd_page = RDD_Log(pagecounts)

rdd_page = MapPartitionsRDD[2] at map at <console>:31


MapPartitionsRDD[2] at map at <console>:31

1. Retrieve the ﬁrst 15 records and print out the result.

In [6]:
for (x <- rdd_page.take(15)) {
   println(x)
}

Log(aa,271_a.C,1,4675)
Log(aa,Category:User_th,1,4770)
Log(aa,Chiron_Elias_Krase,1,4694)
Log(aa,Dassault_rafaele,2,9372)
Log(aa,E.Desv,1,4662)
Log(aa,File:Wiktionary-logo-en.png,1,10752)
Log(aa,Indonesian_Wikipedia,1,4679)
Log(aa,Main_Page,5,266946)
Log(aa,Requests_for_new_languages/Wikipedia_Banyumasan,1,4733)
Log(aa,Special:Contributions/203.144.160.245,1,5812)
Log(aa,Special:Contributions/5.232.61.79,1,5805)
Log(aa,Special:Contributions/Ayarportugal,1,5808)
Log(aa,Special:Contributions/Born2bgratis,1,5812)
Log(aa,Special:ListFiles/Betacommand,1,5035)
Log(aa,Special:ListFiles/Bohdan_p,1,5036)


2. Determine the number of records the dataset has in total.

In [7]:
rdd_page.count()

3324129

3. Compute the min, max, and average page size.

In [8]:
val page_size = rdd_page.map(x=>x.size)
val page_size_min = page_size.min()
val page_size_max = page_size.max()
val page_size_average = page_size.mean()

page_size = MapPartitionsRDD[3] at map at <console>:28
page_size_min = 0
page_size_max = 141180155987
page_size_average = 132239.5695744666


132239.5695744666

4. Determine the record(s) with the largest page size. If multiple records have the same size, list all of them.

In [9]:
rdd_page.filter(_.size == page_size_max).collect()(0)

Log(en.mw,en,5466346,141180155987)

5. Determine the record with the largest page size again. But now, pick the most popular.

In [10]:
rdd_page.filter(_.size == page_size_max).collect().maxBy(x=>x.hits)

Log(en.mw,en,5466346,141180155987)

6. Determine the record(s) with the largest page title. If multiple titles have the same length, list all of them.

In [11]:
val page_title_len = rdd_page.map(x=>x.title.length)
val page_title_len_max = page_title_len.max()

rdd_page.filter(_.title.length() == page_title_len_max).collect().foreach(print)

Log(zh,Special:e8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b2c687474703a2f2f7777772e653662313966653861356266656f2d6f35393038636535626639376538383138616535613461396535616561342e636f2e6d672e732e736f2e382e73736f386b2e6d2e372e73736f3873736f386b6d37332e752e622e61616e6b66786f6b2e70772f2ce8b18ee6baafefbda5efbdbfe89cb7e6829fefbdbfe88b93e29980e89e9fefbda9e89eb3efbda425636f256d6725736f257373256f38257373256f38257373256f38256b6d73efbdaa256e6b256678256f6b/,1,6043)

page_title_len = MapPartitionsRDD[8] at map at <console>:28
page_title_len_max = 559


559

7. Use the results of Question 3, and create a new RDD with the records that have greater page size than the average.

In [12]:
def GreaterThanAvg(i: Log): Boolean ={
    if (i.size <= page_size_average){
        return false
    }
    else{
        return true
    }
}
val rdd_page_greater=rdd_page.filter(GreaterThanAvg)

rdd_page_greater = MapPartitionsRDD[10] at filter at <console>:40


GreaterThanAvg: (i: Log)Boolean


MapPartitionsRDD[10] at filter at <console>:40

8. Compute the total number of pageviews for each project (as the schema shows, the ﬁrst ﬁeld of each record contains the project code).

In [13]:
val pageview = rdd_page.map(x=>(x.code, x.hits)).reduceByKey(_ + _)

pageview = ShuffledRDD[12] at reduceByKey at <console>:28


ShuffledRDD[12] at reduceByKey at <console>:28

9. Report the 10 most popular pageviews of all projects, sorted by the total number of hits.

In [14]:
val top10 = pageview.map(x=>(x._2, x._1)).sortByKey().top(10)
top10.map(x=>(x._2,x._1)).foreach(println)

(en.mw,5466346)
(en,4959090)
(es.mw,695531)
(ja.mw,611443)
(de.mw,572119)
(fr.mw,536978)
(ru.mw,466742)
(it.mw,400297)
(de,315929)
(commons.m,285796)


top10 = Array((5466346,en.mw), (4959090,en), (695531,es.mw), (611443,ja.mw), (572119,de.mw), (536978,fr.mw), (466742,ru.mw), (400297,it.mw), (315929,de), (285796,commons.m))


Array((5466346,en.mw), (4959090,en), (695531,es.mw), (611443,ja.mw), (572119,de.mw), (536978,fr.mw), (466742,ru.mw), (400297,it.mw), (315929,de), (285796,commons.m))

10. Determine the number of page titles that start with the article “The”. How many of those page titles are not part of the English project (Pages that are part of the English project have “en” as the ﬁrst ﬁeld)?

In [15]:
val pagesEN = rdd_page.filter(x => (!x.code.startsWith("en") && x.title.startsWith("The"))).count()

pagesEN = 9128


9128

11. Determine the percentage of pages that have only received a single page view in this one hour of log data.

In [16]:
val onepageview = rdd_page.filter(x => (x.hits == 1)).count().toFloat
val totalpage = rdd_page.count().toFloat
val percentage = onepageview/totalpage

onepageview = 2558332.0
totalpage = 3324129.0
percentage = 0.76962477


0.76962477

12. Determine the number of unique terms appearing in the page titles. Note that in page titles, terms are delimited by “ ” instead of a white space. You can use any number of normalization steps (e.g., lowercasing, removal of non-alphanumeric characters).

In [17]:
val terms = rdd_page.map(x=>x.title.toLowerCase).flatMap(x=>x.split("_")).map(_.replaceAll("[^A-Za-z0-9]", "")).filter(x => x.length >0)

terms = MapPartitionsRDD[23] at filter at <console>:28


MapPartitionsRDD[23] at filter at <console>:28

In [18]:
val distinct_terms = terms.distinct().count()

distinct_terms = 1688528


1688528


13. Determine the most frequently occurring page title term in this dataset.

In [19]:
val res = terms.map(x=>(x,1)).reduceByKey(_ + _).map(x=>(x._2, x._1)).sortByKey().top(1)

res = Array((194407,of))


Array((194407,of))

In [20]:
res.filter(x=>x._2 == "of").take(10)

Array((194407,of))

# Task 2 -Spark SQL

In [21]:
val df = rdd_page.toDF("code","title","hits","size")

df = [code: string, title: string ... 2 more fields]


[code: string, title: string ... 2 more fields]

In [22]:
df.show()

+----+--------------------+----+------+
|code|               title|hits|  size|
+----+--------------------+----+------+
|  aa|             271_a.C|   1|  4675|
|  aa|    Category:User_th|   1|  4770|
|  aa|  Chiron_Elias_Krase|   1|  4694|
|  aa|    Dassault_rafaele|   2|  9372|
|  aa|              E.Desv|   1|  4662|
|  aa|File:Wiktionary-l...|   1| 10752|
|  aa|Indonesian_Wikipedia|   1|  4679|
|  aa|           Main_Page|   5|266946|
|  aa|Requests_for_new_...|   1|  4733|
|  aa|Special:Contribut...|   1|  5812|
|  aa|Special:Contribut...|   1|  5805|
|  aa|Special:Contribut...|   1|  5808|
|  aa|Special:Contribut...|   1|  5812|
|  aa|Special:ListFiles...|   1|  5035|
|  aa|Special:ListFiles...|   1|  5036|
|  aa|Special:ListFiles...|   1|  5032|
|  aa|Special:Log/Md._F...|   1|  5529|
|  aa|Special:Log/MikeL...|   1|  5368|
|  aa|Special:MyLanguag...|   1|  4701|
|  aa|Special:RecentCha...|   1|  6152|
+----+--------------------+----+------+
only showing top 20 rows



3. Compute the min, max, and average page size.

In [23]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import SparkSession._

val sizemax = df.agg(max(col("size"))).collect()(0)(0)
val sizemin = df.agg(min(col("size"))).collect()(0)(0)
val sizemean = df.agg(mean(col("size"))).collect()(0)(0)

sizemax = 141180155987
sizemin = 0
sizemean = 132239.56957446598


132239.56957446598

5. Determine the record with the largest page size again. But now, pick the most popular.

In [24]:
val df_sizemax = df.filter(df("size") === sizemax)
df_sizemax.show()

+-----+-----+-------+------------+
| code|title|   hits|        size|
+-----+-----+-------+------------+
|en.mw|   en|5466346|141180155987|
+-----+-----+-------+------------+



df_sizemax = [code: string, title: string ... 2 more fields]


[code: string, title: string ... 2 more fields]

In [25]:
val ordered = Window.orderBy(df_sizemax.col("hits").desc)

ordered = org.apache.spark.sql.expressions.WindowSpec@7ab5bc71


org.apache.spark.sql.expressions.WindowSpec@7ab5bc71

In [26]:
val ranked = df_sizemax.withColumn("rank", dense_rank.over(ordered))
val maxDf = ranked.filter("rank == 1")
maxDf.show()

+-----+-----+-------+------------+----+
| code|title|   hits|        size|rank|
+-----+-----+-------+------------+----+
|en.mw|   en|5466346|141180155987|   1|
+-----+-----+-------+------------+----+



ranked = [code: string, title: string ... 3 more fields]
maxDf = [code: string, title: string ... 3 more fields]


[code: string, title: string ... 3 more fields]

7. Use the results of Question 3, and create a new RDD with the records that have greater page size than the average.

In [27]:
val df_greater=df.filter(df("size") > sizemean)
df_greater.show()

+------+--------------------+----+------+
|  code|               title|hits|  size|
+------+--------------------+----+------+
|    aa|           Main_Page|   5|266946|
|ace.mw|                 ace|  31|827168|
|    af|                1859|   4|219540|
|    af|          18_Oktober|   4|264724|
|    af|                1941|   4|256344|
|    af|                2016|   5|215498|
|    af|          4_Januarie|   4|268828|
|    af|         Afrika-unie|   1|172078|
|    af|             Big_Ben|  13|136201|
|    af|    Comrades-maraton|   1|155180|
|    af|     Dmitri_Medwedef|   2|141328|
|    af|               Elsas|   4|319408|
|    af|              Engels|   2|182375|
|    af|         Erich_Fromm|   4|215612|
|    af|            Filosoof|   2|134400|
|    af|                GNTA|  77|511277|
|    af|Gebruiker:Aliwal2012|   2|359320|
|    af|      Gebruiker:JCIV|   2|268216|
|    af|     Gebruiker:Morne|   3|991701|
|    af|   Gebruiker:Naudefj|   3|730849|
+------+--------------------+----+

df_greater = [code: string, title: string ... 2 more fields]


[code: string, title: string ... 2 more fields]

12. Determine the number of unique terms appearing in the page titles. Note that in page titles, terms are delimited by “ ” instead of a white space. You can use any number of normalization steps (e.g., lowercasing, removal of non-alphanumeric characters).

In [28]:
val df1 = df.select(explode(split(lower(col("title")),"_")))

df1 = [col: string]


[col: string]

In [29]:
val df_term = df1.withColumn("col",regexp_replace($"col", "[^A-Za-z0-9]", "")).filter(length($"col")>0)

df_term = [col: string]


[col: string]

In [30]:
val num_term = df_term.distinct.count()

num_term = 1688528


1688528

13.Determine the most frequently occurring page title term in this dataset.

In [31]:
val df_term_new = df_term.groupBy("col").count()

df_term_new = [col: string, count: bigint]


[col: string, count: bigint]

In [32]:
df_term_new.orderBy($"count".desc).first

[of,194407]